In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext(appName = 'DicomDF')
sqlContext = SQLContext(sc)

In [2]:
from pyqae.med import dicom_paths_to_df
from pyqae.utils import local_read_depth
import numpy as np

In [3]:
test_files = local_read_depth('../test/resources', 1, inc_parent = False)

In [5]:
dicom_df = dicom_paths_to_df(test_files)
dicom_df.apply(lambda x: type(x.values[0])).value_counts()

<class 'bytes'>    90
<class 'str'>      50
<class 'int'>      15
<class 'float'>    14
<class 'list'>      6
dtype: int64

In [6]:
# not all types are supported by spark sql
valid_cols = dicom_df.apply(lambda x: type(x.values[0]) in [type(''), int, float])
valid_col_df = dicom_df[valid_cols.index[np.where(valid_cols)[0]]]
valid_col_df.sample(1)

,Accession Number,Acquisition Date,Acquisition Number,Acquisition Time,Additional Patient History,Bits Allocated,Bits Stored,Body Part Examined,Code Meaning,Code Value,...,Specific Character Set,Study Date,Study Description,Study Instance UID,Study Time,Table Height,Window Center,Window Width,X-Ray Tube Current,DICOMPath4Q
0,7731341270494673,19890810,1,113713,F/U LUNG CA.,16,16,LUNG,Retain Safe Private Option,113111,...,ISO_IR 100,19890810,CT CHEST,1.3.6.1.4.1.14519.5.2.1.6450.4012.206382517630...,113536,139.5,40.0,400.0,500,../test/resources/dicom/10-060.dcm


In [7]:
tdf = sqlContext.createDataFrame(valid_col_df)
tdf.registerTempTable('DICOMSingleTable')
sqlContext.sql("""SELECT * FROM DICOMSingleTable LIMIT 1""").toPandas()

,Accession Number,Acquisition Date,Acquisition Number,Acquisition Time,Additional Patient History,Bits Allocated,Bits Stored,Body Part Examined,Code Meaning,Code Value,...,Specific Character Set,Study Date,Study Description,Study Instance UID,Study Time,Table Height,Window Center,Window Width,X-Ray Tube Current,DICOMPath4Q
0,7731341270494673,19890810,1,113713,F/U LUNG CA.,16,16,LUNG,Retain Safe Private Option,113111,...,ISO_IR 100,19890810,CT CHEST,1.3.6.1.4.1.14519.5.2.1.6450.4012.206382517630...,113536,139.5,40.0,400.0,500,../test/resources/dicom/10-060.dcm


In [8]:
sc.stop()